In [ ]:
# Test XAI with scaling
import numpy as np
import torch
import joblib
import pandas as pd

# Load scaler
scaler = joblib.load('../step1_lstm_xai/scaler.joblib')

# Get one alert from your data
alert_features = df.iloc[0][FEATURE_NAMES].values

# Scale it
alert_scaled = scaler.transform(alert_features.reshape(1, -1))

# Create tensor
alert_tensor = torch.tensor(alert_scaled, dtype=torch.float32).unsqueeze(0).to(device)

# Get prediction
model.eval()
with torch.no_grad():
    output = model(alert_tensor)
    probs = torch.softmax(output, dim=1)[0]
    print(f"Prediction: {output.argmax(dim=1).item()}")
    print(f"Confidence: {probs[output.argmax(dim=1)].item():.4f}")

# Compute attributions
from captum.attr import DeepLift
dl = DeepLift(model)
baseline = torch.zeros_like(alert_tensor)
attributions = dl.attribute(alert_tensor, baselines=baseline, target=output.argmax(dim=1).item())

print("\nTop 5 Feature Importances:")
attr_array = attributions.squeeze().detach().cpu().numpy()
sorted_indices = np.argsort(-np.abs(attr_array))[:5]
for idx in sorted_indices:
    print(f"  {FEATURE_NAMES[idx]}: {attr_array[idx]:.4f}")

/Users/luwaga/anaconda3/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


NameError: name 'df' is not defined